In [1]:
import numpy as np
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import time
import shutil
from tqdm import tqdm
import os
import pandas as pd
import cv2
import matplotlib.pyplot as plt

### This will modify the training images only (only the images with annotations, so dataset 1 and 2, or wsi images # 1,2,3,4.

In [2]:
original_image_src = r"\\fatherserverdw\Kevin\hubmap\train" #all 1622 images
original_bv_mask_src = r"\\fatherserverdw\Kevin\hubmap\unet++\masks\blood_vessel"
original_glom_mask_src = r"\\fatherserverdw\Kevin\hubmap\unet++\masks\glomerulus"
tile_meta_src = r"\\fatherserverdw\Kevin\hubmap\yolov8_v2\obj_detect_bv_glo.xlsx" #r"\\fatherserverdw\Kevin\hubmap\tile_meta.csv"
annotation_meta_src = r"\\fatherserverdw\Kevin\hubmap\annotation_meta.xlsx"

### Code to create overlapping images for all WSI's:

In [3]:
for wsi_idx in tqdm(range(1,5)):
    tile_df = pd.read_excel(tile_meta_src)
    tmp_df = pd.read_excel(annotation_meta_src)
    supervised_tile_df = tile_df[tile_df["source_wsi"] == wsi_idx]
    supervised_tile_ids = np.unique(supervised_tile_df.id.tolist()) #list of picture ids that are in source_wsi = 1
    supervised_tile_df = supervised_tile_df.sort_values(by=['i', 'j'])
    image_src = r"\\fatherserverdw\Kevin\hubmap\train"
    bv_mask_src= r"\\fatherserverdw\Kevin\hubmap\unet++\masks\blood_vessel_dilated"
    glom_mask_src= r"\\fatherserverdw\Kevin\hubmap\unet++\masks\glomerulus_dilated"
    wsi_tiles = [os.path.join(image_src,x + ".tif") for x in supervised_tile_ids]
    wsi_bv_masks = [os.path.join(bv_mask_src,x + ".png") for x in supervised_tile_ids]
    glom_df = tmp_df[tmp_df["type"] == "glomerulus"]
    glom_df_ids = glom_df["id"].tolist()
    glom_ids = [x for x in supervised_tile_ids if x in glom_df_ids]
    print(len(glom_ids)) #different for each wsi
    wsi_glom_masks = [os.path.join(glom_mask_src,x + ".png") for x in glom_ids]
    image_paths = wsi_tiles # List of image paths (.tif)
    bv_mask_paths = wsi_bv_masks   # List of corresponding mask paths (.png)
    gl_mask_paths = wsi_glom_masks
    images = {} #dictionary
    bv_masks = {}
    gl_masks = {}

    # Load all the images and masks into memory
    for image_path, bv_mask_path in tqdm(zip(image_paths, bv_mask_paths),total = len(image_paths)):
        image_name = os.path.splitext(os.path.basename(image_path))[0]
        bv_mask_name = os.path.splitext(os.path.basename(bv_mask_path))[0]
        images[image_name] = cv2.imread(image_path)
        bv_masks[bv_mask_name] = cv2.imread(bv_mask_path,0)
    for gl_mask_path in tqdm(gl_mask_paths, total = len(gl_mask_paths)):
        gl_mask_name = os.path.splitext(os.path.basename(gl_mask_path))[0]
        gl_masks[gl_mask_name] = cv2.imread(gl_mask_path,0)
    # Create a directory to store the overlap images and masks
    image_save_dir = r"\\fatherserverdw\Kevin\hubmap\train_overlap\images"
    bv_mask_save_dir = r"\\fatherserverdw\Kevin\hubmap\train_overlap\masks\blood_vessel"
    gl_mask_save_dir = r"\\fatherserverdw\Kevin\hubmap\train_overlap\masks\glomerulus"
    overlap_size = 256
    supervised_tile_df = supervised_tile_df.groupby(["id"]).agg({"i":"first","j":"first"})
    supervised_tile_df = supervised_tile_df.reset_index()
    supervised_tile_df = supervised_tile_df.sort_values(by=['i', 'j'])

    # Iterate over each row in the DataFrame, Repeat this for each wsi# 1~4:
    for index, row in tqdm(supervised_tile_df.iterrows(), total = supervised_tile_df.shape[0]):
        #Get the image and mask names
        image_name = row['id']
        bv_mask_name = image_name
        if image_name in list(gl_masks.keys()):
            gl_mask_name = image_name
            gl_mask_exists = True
        else:
            gl_mask_exists = False
        #Get the coordinates of the current image
        x = row['i']
        y = row['j']

        #Returns id of all neighboring images:
        left_image_name = supervised_tile_df[(supervised_tile_df['i'] == x-512) & (supervised_tile_df['j'] == y)]['id'].values
        top_left_image_name = supervised_tile_df[(supervised_tile_df['i'] == x-512) & (supervised_tile_df['j'] == y-512)]['id'].values
        top_image_name = supervised_tile_df[(supervised_tile_df['i'] == x) & (supervised_tile_df['j'] == y-512)]['id'].values
        top_right_image_name = supervised_tile_df[(supervised_tile_df['i'] == x+512) & (supervised_tile_df['j'] == y-512)]['id'].values
        right_image_name = supervised_tile_df[(supervised_tile_df['i'] == x+512) & (supervised_tile_df['j'] == y)]['id'].values
        bot_right_image_name = supervised_tile_df[(supervised_tile_df['i'] == x+512) & (supervised_tile_df['j'] == y+512)]['id'].values
        bot_image_name = supervised_tile_df[(supervised_tile_df['i'] == x) & (supervised_tile_df['j'] == y+512)]['id'].values
        bot_left_image_name = supervised_tile_df[(supervised_tile_df['i'] == x-512) & (supervised_tile_df['j'] == y+512)]['id'].values

        #Get the image and mask array for the current tile
        image = images[image_name]
        bv_mask = bv_masks[bv_mask_name]

        #Save original images first:
        overlap_image_path = os.path.join(image_save_dir, f"{image_name}.tif")
        overlap_mask_path = os.path.join(bv_mask_save_dir, f"{bv_mask_name}.tif")
        cv2.imwrite(overlap_image_path, image)
        cv2.imwrite(overlap_mask_path, bv_mask)
        if gl_mask_exists:
            overlap_gl_mask_path = os.path.join(gl_mask_save_dir, f"{gl_mask_name}.tif")
            gl_mask = gl_masks[gl_mask_name]
            cv2.imwrite(overlap_gl_mask_path, gl_mask)

        image_name_list = list(images.keys())
        gl_mask_name_list = list(gl_masks.keys())
        #Now check if the neighboring images exist, by checking if image names are in the entire image list for each quadrant overlap:
        # For top left overlap
        if left_image_name in image_name_list and top_image_name in image_name_list and top_left_image_name in image_name_list:
            final_top_left_overlap_image = np.zeros((512, 512, 3))
            final_top_left_overlap_bv_mask = np.zeros((512, 512))
            left_image = images[left_image_name[0]]
            top_left_image = images[top_left_image_name[0]]
            top_image = images[top_image_name[0]]

            # slicing goes [ymin:ymax,xmin:xmax]
            top_left_overlap = top_left_image[256:512,256:512,:]
            top_left_bv_mask_overlap = bv_masks[top_left_image_name[0]][256:512,256:512]

            top_overlap = top_image[256:512,0:256,:]
            top_bv_mask_overlap = bv_masks[top_image_name[0]][256:512,0:256]

            left_overlap = left_image[0:256,256:512,:]
            left_bv_mask_overlap = bv_masks[left_image_name[0]][0:256,256:512]

            original_top_left_image = image[0:256,0:256,:]
            original_top_left_bv_mask = bv_mask[0:256,0:256]

            final_top_left_overlap_image[256:, 256:,:] = original_top_left_image
            final_top_left_overlap_image[256:, :256,:] = left_overlap
            final_top_left_overlap_image[:256, 256:,:] = top_overlap
            final_top_left_overlap_image[:256, :256,:] = top_left_overlap

            final_top_left_overlap_bv_mask[256:, 256:] = original_top_left_bv_mask
            final_top_left_overlap_bv_mask[256:, :256] = left_bv_mask_overlap
            final_top_left_overlap_bv_mask[:256, 256:] = top_bv_mask_overlap
            final_top_left_overlap_bv_mask[:256, :256] = top_left_bv_mask_overlap

            overlap_image_path = os.path.join(image_save_dir, f"{image_name}_top_left.tif")
            overlap_mask_path = os.path.join(bv_mask_save_dir, f"{bv_mask_name}_top_left.tif")
            cv2.imwrite(overlap_image_path, final_top_left_overlap_image.astype(np.uint8))
            cv2.imwrite(overlap_mask_path, final_top_left_overlap_bv_mask.astype(np.uint8))
            if gl_mask_exists:
                if left_image_name in gl_mask_name_list and top_image_name in gl_mask_name_list and top_left_image_name in gl_mask_name_list:
                    final_top_left_overlap_gl_mask = np.zeros((512, 512))
                    top_left_gl_mask_overlap = gl_masks[top_left_image_name[0]][256:,256:]
                    top_gl_mask_overlap = gl_masks[top_image_name[0]][256:,:256]
                    left_gl_mask_overlap = gl_masks[left_image_name[0]][:256,256:]
                    original_top_left_gl_mask = gl_mask[:256,:256]

                    final_top_left_overlap_gl_mask[256:, 256:] = original_top_left_gl_mask
                    final_top_left_overlap_gl_mask[256:, :256] = left_gl_mask_overlap
                    final_top_left_overlap_gl_mask[:256, 256:] = top_gl_mask_overlap
                    final_top_left_overlap_gl_mask[:256, :256] = top_left_gl_mask_overlap

                    overlap_gl_mask_path = os.path.join(gl_mask_save_dir, f"{gl_mask_name}_top_left.tif")
                    cv2.imwrite(overlap_gl_mask_path, final_top_left_overlap_gl_mask.astype(np.uint8))
        # For top right overlap
        if right_image_name in image_name_list and top_right_image_name in image_name_list and top_image_name in image_name_list:
            final_top_right_overlap_image = np.zeros((512, 512, 3))
            final_top_right_overlap_bv_mask = np.zeros((512, 512))
            right_image = images[right_image_name[0]]
            top_right_image = images[top_right_image_name[0]]
            top_image = images[top_image_name[0]]

            right_overlap = right_image[:256,:256,:]
            right_bv_mask_overlap = bv_masks[right_image_name[0]][:256,:256]

            top_right_overlap = top_right_image[256:,:256,:]
            top_right_bv_mask_overlap = bv_masks[top_right_image_name[0]][256:,:256]

            top_overlap = top_image[256:,256:,:]
            top_bv_mask_overlap = bv_masks[top_image_name[0]][256:,256:]

            original_top_right_image = image[:256,256:,:]
            original_top_right_bv_mask = bv_mask[:256,256:]

            final_top_right_overlap_image[256:,256:,:] = right_overlap
            final_top_right_overlap_image[256:,:256,:] = original_top_right_image
            final_top_right_overlap_image[:256, 256:,:] = top_right_overlap
            final_top_right_overlap_image[:256,:256,:] = top_overlap

            final_top_right_overlap_bv_mask[256:,256:] = right_bv_mask_overlap
            final_top_right_overlap_bv_mask[256:,:256] = original_top_right_bv_mask
            final_top_right_overlap_bv_mask[:256, 256:] = top_right_bv_mask_overlap
            final_top_right_overlap_bv_mask[:256,:256] = top_bv_mask_overlap

            overlap_image_path = os.path.join(image_save_dir, f"{image_name}_top_right.tif")
            overlap_mask_path = os.path.join(bv_mask_save_dir, f"{bv_mask_name}_top_right.tif")
            cv2.imwrite(overlap_image_path, final_top_right_overlap_image.astype(np.uint8))
            cv2.imwrite(overlap_mask_path, final_top_right_overlap_bv_mask.astype(np.uint8))
            if gl_mask_exists:
                if right_image_name in gl_mask_name_list and top_right_image_name in gl_mask_name_list and top_image_name in gl_mask_name_list:
                    final_top_right_overlap_gl_mask = np.zeros((512, 512))

                    right_gl_mask_overlap = gl_masks[right_image_name[0]][:256,:256]
                    top_right_gl_mask_overlap = gl_masks[top_right_image_name[0]][256:,:256]
                    top_gl_mask_overlap = gl_masks[top_image_name[0]][256:,256:]
                    original_top_right_gl_mask = gl_mask[:256,256:]

                    final_top_right_overlap_gl_mask[256:,256:] = right_gl_mask_overlap
                    final_top_right_overlap_gl_mask[256:,:256] = original_top_right_gl_mask
                    final_top_right_overlap_gl_mask[:256, 256:] = top_right_gl_mask_overlap
                    final_top_right_overlap_gl_mask[:256,:256] = top_gl_mask_overlap

                    overlap_gl_mask_path = os.path.join(gl_mask_save_dir, f"{gl_mask_name}_top_right.tif")
                    cv2.imwrite(overlap_gl_mask_path, final_top_right_overlap_gl_mask.astype(np.uint8))

        # For bot right overlap
        if right_image_name in image_name_list and bot_right_image_name in image_name_list and bot_image_name in image_name_list:
            final_bot_right_overlap_image = np.zeros((512, 512, 3))
            final_bot_right_overlap_bv_mask = np.zeros((512, 512))
            right_image = images[right_image_name[0]]
            bot_right_image = images[bot_right_image_name[0]]
            bot_image = images[bot_image_name[0]]

            right_overlap = right_image[256:512,0:256,:]
            right_bv_mask_overlap = bv_masks[right_image_name[0]][256:512,0:256]

            bot_right_overlap = bot_right_image[0:256,0:256,:]
            bot_right_bv_mask_overlap = bv_masks[bot_right_image_name[0]][0:256,0:256]

            bot_overlap = bot_image[0:256,256:512,:]
            bot_bv_mask_overlap = bv_masks[bot_image_name[0]][0:256,256:512]

            original_bot_right_image = image[256:512,256:512,:]
            original_bot_right_bv_mask = bv_mask[256:512,256:512]

            final_bot_right_overlap_image[256:,256:,] = bot_right_overlap
            final_bot_right_overlap_image[256:,:256,:] = bot_overlap
            final_bot_right_overlap_image[:256, 256:,:] = right_overlap
            final_bot_right_overlap_image[:256,:256,:] = original_bot_right_image

            final_bot_right_overlap_bv_mask[256:,256:] = bot_right_bv_mask_overlap
            final_bot_right_overlap_bv_mask[256:,:256] = bot_bv_mask_overlap
            final_bot_right_overlap_bv_mask[:256, 256:] = right_bv_mask_overlap
            final_bot_right_overlap_bv_mask[:256,:256] = original_bot_right_bv_mask

            overlap_image_path = os.path.join(image_save_dir, f"{image_name}_bot_right.tif")
            overlap_mask_path = os.path.join(bv_mask_save_dir, f"{bv_mask_name}_bot_right.tif")
            cv2.imwrite(overlap_image_path, final_bot_right_overlap_image.astype(np.uint8))
            cv2.imwrite(overlap_mask_path, final_bot_right_overlap_bv_mask.astype(np.uint8))

            if gl_mask_exists:
                if right_image_name in gl_mask_name_list and bot_right_image_name in gl_mask_name_list and bot_image_name in gl_mask_name_list:
                    final_bot_right_overlap_gl_mask = np.zeros((512, 512))

                    right_gl_mask_overlap = gl_masks[right_image_name[0]][256:,:256]
                    bot_right_gl_mask_overlap = gl_masks[bot_right_image_name[0]][:256,:256]
                    bot_gl_mask_overlap = gl_masks[bot_image_name[0]][:256,256:]
                    original_bot_right_gl_mask = gl_mask[256:,256:]

                    final_bot_right_overlap_gl_mask[256:,256:] = bot_right_gl_mask_overlap
                    final_bot_right_overlap_gl_mask[256:,:256] = bot_gl_mask_overlap
                    final_bot_right_overlap_gl_mask[:256, 256:] = right_gl_mask_overlap
                    final_bot_right_overlap_gl_mask[:256,:256] = original_bot_right_gl_mask

                    overlap_gl_mask_path = os.path.join(gl_mask_save_dir, f"{gl_mask_name}_bot_right.tif")
                    cv2.imwrite(overlap_gl_mask_path, final_bot_right_overlap_gl_mask.astype(np.uint8))

         # For bot left overlap:
        if left_image_name in image_name_list and bot_image_name in image_name_list and bot_left_image_name in image_name_list:
            final_bot_left_overlap_image = np.zeros((512, 512, 3))
            final_bot_left_overlap_bv_mask = np.zeros((512, 512))
            left_image = images[left_image_name[0]]
            bot_left_image = images[bot_left_image_name[0]]
            bot_image = images[bot_image_name[0]]

            # slicing goes [ymin:ymax,xmin:xmax]
            bot_left_overlap = bot_left_image[:256,256:,:]
            bot_left_bv_mask_overlap = bv_masks[bot_left_image_name[0]][:256,256:]

            bot_overlap = bot_image[:256,:256,:]
            bot_bv_mask_overlap = bv_masks[bot_image_name[0]][:256,:256]

            left_overlap = left_image[256:,256:,:]
            left_bv_mask_overlap = bv_masks[left_image_name[0]][256:,256:]

            original_bot_left_image = image[256:,:256,:]
            original_bot_left_bv_mask = bv_mask[256:,:256]

            final_bot_left_overlap_image[256:,256:,:] = bot_overlap
            final_bot_left_overlap_image[256:, :256,:] = bot_left_overlap
            final_bot_left_overlap_image[:256,256:,:] = original_bot_left_image
            final_bot_left_overlap_image[:256,:256,:] = left_overlap

            final_bot_left_overlap_bv_mask[256:,256:] = bot_bv_mask_overlap
            final_bot_left_overlap_bv_mask[256:, :256] = bot_left_bv_mask_overlap
            final_bot_left_overlap_bv_mask[:256,256:] = original_bot_left_bv_mask
            final_bot_left_overlap_bv_mask[:256,:256] = left_bv_mask_overlap

            overlap_image_path = os.path.join(image_save_dir, f"{image_name}_bot_left.tif")
            overlap_mask_path = os.path.join(bv_mask_save_dir, f"{bv_mask_name}_bot_left.tif")
            cv2.imwrite(overlap_image_path, final_bot_left_overlap_image.astype(np.uint8))
            cv2.imwrite(overlap_mask_path, final_bot_left_overlap_bv_mask.astype(np.uint8))
            if gl_mask_exists:
                if left_image_name in gl_mask_name_list and bot_image_name in gl_mask_name_list and bot_left_image_name in gl_mask_name_list:
                    final_bot_left_overlap_gl_mask = np.zeros((512, 512))

                    bot_left_gl_mask_overlap = gl_masks[bot_left_image_name[0]][:256,256:]
                    bot_gl_mask_overlap = gl_masks[bot_image_name[0]][:256,:256]
                    left_gl_mask_overlap = gl_masks[left_image_name[0]][256:,256:]
                    original_bot_left_gl_mask = gl_mask[256:,:256]

                    final_bot_left_overlap_gl_mask[256:,256:] = bot_gl_mask_overlap
                    final_bot_left_overlap_gl_mask[256:, :256] = bot_left_gl_mask_overlap
                    final_bot_left_overlap_gl_mask[:256,256:] = original_bot_left_gl_mask
                    final_bot_left_overlap_gl_mask[:256,:256] = left_gl_mask_overlap

                    overlap_gl_mask_path = os.path.join(gl_mask_save_dir, f"{gl_mask_name}_bot_left.tif")
                    cv2.imwrite(overlap_gl_mask_path, final_bot_left_overlap_gl_mask.astype(np.uint8))
        else:
            print("No neighboring images found for {}, skipping to next image".format(image_name))
            continue


  0%|          | 0/4 [00:00<?, ?it/s]

87



100%|██████████| 496/496 [02:24<00:00,  3.43it/s]

100%|██████████| 87/87 [00:01<00:00, 46.50it/s]

  0%|          | 0/496 [00:00<?, ?it/s]C:\Users\Kevin\AppData\Local\Temp\ipykernel_2020\2702430439.py:84: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if left_image_name in image_name_list and top_image_name in image_name_list and top_left_image_name in image_name_list:
C:\Users\Kevin\AppData\Local\Temp\ipykernel_2020\2702430439.py:134: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if right_image_name in image_name_list and top_right_image_name in image_name_list and top_image_name in image_name_list:
C:\Users\Kevin\AppData\Local\Temp\ipykernel_2020\2702430439.py:237: DeprecationWarning: The truth value o

No neighboring images found for 5631a47d5b0c, skipping to next image


C:\Users\Kevin\AppData\Local\Temp\ipykernel_2020\2702430439.py:185: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if right_image_name in image_name_list and bot_right_image_name in image_name_list and bot_image_name in image_name_list:

  0%|          | 2/496 [00:01<06:04,  1.35it/s]

...

No neighboring images found for 620901cb76b3, skipping to next image


 99%|█████████▊| 267/271 [02:34<00:01,  3.19it/s]

No neighboring images found for 989936714cf5, skipping to next image



100%|██████████| 4/4 [22:26<00:00, 336.68s/it]

No neighboring images found for dd4a8ba579c0, skipping to next image
No neighboring images found for 3b0d08f3e37a, skipping to next image
No neighboring images found for c0073ae5f1f6, skipping to next image
